3.1 Pricing Strategy A/B Test Simulation

In [ ]:
# Import libraries for statistical testing
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower
import random

# Simulate an A/B test for pricing strategies
np.random.seed(42)

# Function to simulate customer behavior under different pricing
def simulate_customer_behavior(price, base_conversion_rate=0.05, price_elasticity=-0.005):
    # Conversion rate decreases as price increases
    conversion_rate = base_conversion_rate + price_elasticity * (price - 100)
    # Ensure conversion rate is between 0 and 1
    conversion_rate = max(0.01, min(0.2, conversion_rate))
    # Simulate conversion (1 = purchase, 0 = no purchase)
    return np.random.binomial(1, conversion_rate)

# Set up the experiment
control_price = 100  # Original price
treatment_price = 90  # Discounted price

# Sample size calculation
effect_size = 0.1  # Expected effect size (10% increase in conversion)
alpha = 0.05  # Significance level
power = 0.8  # Statistical power

# Calculate required sample size
power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha)
sample_size = int(np.ceil(sample_size))

print(f"Required sample size per group: {sample_size}")

# Simulate the experiment with a larger sample for demonstration
sample_size = max(sample_size, 1000)
total_customers = sample_size * 2

# Create experiment data
experiment_data = pd.DataFrame({
    'customer_id': range(total_customers),
    'group': ['control' if i < sample_size else 'treatment' for i in range(total_customers)]
})

# Apply pricing and simulate conversions
experiment_data['price'] = experiment_data['group'].map({'control': control_price, 'treatment': treatment_price})
experiment_data['converted'] = experiment_data.apply(
    lambda row: simulate_customer_behavior(row['price']), axis=1
)

# Calculate revenue
experiment_data['revenue'] = experiment_data['converted'] * experiment_data['price']

# Analyze results
results_by_group = experiment_data.groupby('group').agg({
    'customer_id': 'count',
    'converted': ['sum', 'mean'],
    'revenue': ['sum', 'mean']
}).reset_index()

# Flatten multi-level columns
results_by_group.columns = ['group', 'customers', 'conversions', 'conversion_rate', 'total_revenue', 'average_revenue']

print("\nA/B Test Results:")
display(results_by_group)

# Statistical testing for conversion rate
control_conversions = experiment_data[experiment_data['group'] == 'control']['converted']
treatment_conversions = experiment_data[experiment_data['group'] == 'treatment']['converted']

# Chi-square test for conversion rates
contingency_table = pd.crosstab(experiment_data['group'], experiment_data['converted'])
chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)

print(f"\nChi-square test for conversion rates:")
print(f"Chi2 value: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Statistically significant difference: {p_value < alpha}")

# T-test for revenue
t_stat, p_value_revenue = stats.ttest_ind(
    experiment_data[experiment_data['group'] == 'control']['revenue'],
    experiment_data[experiment_data['group'] == 'treatment']['revenue']
)

print(f"\nT-test for revenue:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value_revenue:.4f}")
print(f"Statistically significant difference: {p_value_revenue < alpha}")

# Calculate effect sizes
control_conv_rate = results_by_group[results_by_group['group'] == 'control']['conversion_rate'].values[0]
treatment_conv_rate = results_by_group[results_by_group['group'] == 'treatment']['conversion_rate'].values[0]
relative_lift_conv = (treatment_conv_rate - control_conv_rate) / control_conv_rate * 100

control_revenue = results_by_group[results_by_group['group'] == 'control']['total_revenue'].values[0]
treatment_revenue = results_by_group[results_by_group['group'] == 'treatment']['total_revenue'].values[0]
relative_lift_revenue = (treatment_revenue - control_revenue) / control_revenue * 100

print(f"\nRelative lift in conversion rate: {relative_lift_conv:.2f}%")
print(f"Relative lift in total revenue: {relative_lift_revenue:.2f}%")

# Visualize results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.barplot(x='group', y='conversion_rate', data=results_by_group, palette='viridis')
plt.title('Conversion Rate by Group')
plt.xlabel('Group')
plt.ylabel('Conversion Rate')
plt.ylim(0, max(results_by_group['conversion_rate']) * 1.2)

plt.subplot(1, 2, 2)
sns.barplot(x='group', y='total_revenue', data=results_by_group, palette='viridis')
plt.title('Total Revenue by Group')
plt.xlabel('Group')
plt.ylabel('Total Revenue')
plt.ylim(0, max(results_by_group['total_revenue']) * 1.2)

plt.tight_layout()
plt.show()

print("\nPricing strategy A/B test simulation complete.")

3.2 Delivery Options A/B Test Framework

In [ ]:
# Simulate an A/B test for different delivery options
np.random.seed(43)

# Function to simulate customer satisfaction based on delivery time
def simulate_satisfaction(delivery_time, expected_time=15):
    # Base satisfaction score (1-5)
    base_score = 4.0
    
    # Adjust based on delivery time relative to expectation
    time_factor = (expected_time - delivery_time) / expected_time
    
    # Add some random noise
    noise = np.random.normal(0, 0.5)
    
    # Calculate final score
    score = base_score + time_factor + noise
    
    # Ensure score is between 1 and 5
    return max(1, min(5, score))

# Set up the experiment
control_delivery = 'Standard'  # 5-15 days
treatment_delivery = 'Express'  # 2-7 days

# Simulate delivery times
def get_delivery_time(delivery_option):
    if delivery_option == 'Standard':
        return np.random.randint(5, 16)  # 5-15 days
    else:  # Express
        return np.random.randint(2, 8)   # 2-7 days

# Create experiment data
sample_size = 500  # per group
total_customers = sample_size * 2

delivery_experiment = pd.DataFrame({
    'customer_id': range(total_customers),
    'group': ['control' if i < sample_size else 'treatment' for i in range(total_customers)]
})

# Apply delivery options
delivery_experiment['delivery_option'] = delivery_experiment['group'].map({
    'control': control_delivery, 
    'treatment': treatment_delivery
})

# Simulate delivery times and satisfaction
delivery_experiment['delivery_time'] = delivery_experiment['delivery_option'].apply(get_delivery_time)
delivery_experiment['satisfaction'] = delivery_experiment['delivery_time'].apply(simulate_satisfaction)

# Add cost information
delivery_experiment['delivery_cost'] = delivery_experiment['delivery_option'].map({
    'Standard': 10, 
    'Express': 20
})

# Analyze results
delivery_results = delivery_experiment.groupby('group').agg({
    'customer_id': 'count',
    'delivery_time': ['mean', 'std'],
    'satisfaction': ['mean', 'std'],
    'delivery_cost': 'mean'
}).reset_index()

# Flatten multi-level columns
delivery_results.columns = ['group', 'customers', 'avg_delivery_time', 'std_delivery_time', 
                           'avg_satisfaction', 'std_satisfaction', 'avg_delivery_cost']

print("\nDelivery Options A/B Test Results:")
display(delivery_results)

# Statistical testing for satisfaction scores
control_satisfaction = delivery_experiment[delivery_experiment['group'] == 'control']['satisfaction']
treatment_satisfaction = delivery_experiment[delivery_experiment['group'] == 'treatment']['satisfaction']

t_stat, p_value = stats.ttest_ind(control_satisfaction, treatment_satisfaction)

print(f"\nT-test for satisfaction scores:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Statistically significant difference: {p_value < 0.05}")

# Calculate effect size (Cohen's d)
cohens_d = (treatment_satisfaction.mean() - control_satisfaction.mean()) / np.sqrt(
    (treatment_satisfaction.std() ** 2 + control_satisfaction.std() ** 2) / 2
)

print(f"Effect size (Cohen's d): {cohens_d:.4f}")

# Cost-benefit analysis
avg_control_satisfaction = delivery_results[delivery_results['group'] == 'control']['avg_satisfaction'].values[0]
avg_treatment_satisfaction = delivery_results[delivery_results['group'] == 'treatment']['avg_satisfaction'].values[0]
satisfaction_improvement = avg_treatment_satisfaction - avg_control_satisfaction

control_cost = delivery_results[delivery_results['group'] == 'control']['avg_delivery_cost'].values[0]
treatment_cost = delivery_results[delivery_results['group'] == 'treatment']['avg_delivery_cost'].values[0]
cost_increase = treatment_cost - control_cost

cost_per_satisfaction_point = cost_increase / satisfaction_improvement if satisfaction_improvement > 0 else float('inf')

print(f"\nCost-benefit analysis:")
print(f"Satisfaction improvement: {satisfaction_improvement:.2f} points")
print(f"Cost increase: ${cost_increase:.2f}")
print(f"Cost per satisfaction point: ${cost_per_satisfaction_point:.2f}")

# Visualize results
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x='group', y='delivery_time', data=delivery_experiment, palette='viridis')
plt.title('Delivery Time by Group')
plt.xlabel('Group')
plt.ylabel('Delivery Time (days)')

plt.subplot(2, 2, 2)
sns.boxplot(x='group', y='satisfaction', data=delivery_experiment, palette='viridis')
plt.title('Customer Satisfaction by Group')
plt.xlabel('Group')
plt.ylabel('Satisfaction Score (1-5)')

plt.subplot(2, 2, 3)
sns.scatterplot(x='delivery_time', y='satisfaction', hue='group', data=delivery_experiment, palette='viridis')
plt.title('Satisfaction vs. Delivery Time')
plt.xlabel('Delivery Time (days)')
plt.ylabel('Satisfaction Score (1-5)')

plt.subplot(2, 2, 4)
sns.barplot(x='group', y='avg_delivery_cost', data=delivery_results, palette='viridis')
plt.title('Average Delivery Cost by Group')
plt.xlabel('Group')
plt.ylabel('Average Cost ($)')

plt.tight_layout()
plt.show()

print("\nDelivery options A/B test simulation complete.")

3.3 A/B Testing Framework Implementation

In [ ]:
# Create a general A/B testing framework
class ABTestFramework:
    def __init__(self, name, metric_name, alpha=0.05, power=0.8, effect_size=0.1):
        self.name = name
        self.metric_name = metric_name
        self.alpha = alpha
        self.power = power
        self.effect_size = effect_size
        self.control_data = []
        self.treatment_data = []
        
    def calculate_sample_size(self):
        """Calculate required sample size per group"""
        power_analysis = TTestIndPower()
        sample_size = power_analysis.solve_power(
            effect_size=self.effect_size, 
            power=self.power, 
            alpha=self.alpha
        )
        return int(np.ceil(sample_size))
    
    def add_data(self, group, value):
        """Add observation to the appropriate group"""
        if group == 'control':
            self.control_data.append(value)
        elif group == 'treatment':
            self.treatment_data.append(value)
        else:
            raise ValueError("Group must be 'control' or 'treatment'")
    
    def get_summary_stats(self):
        """Calculate summary statistics for both groups"""
        control_mean = np.mean(self.control_data) if self.control_data else 0
        treatment_mean = np.mean(self.treatment_data) if self.treatment_data else 0
        control_std = np.std(self.control_data) if len(self.control_data) > 1 else 0
        treatment_std = np.std(self.treatment_data) if len(self.treatment_data) > 1 else 0
        
        return {
            'control_n': len(self.control_data),
            'treatment_n': len(self.treatment_data),
            'control_mean': control_mean,
            'treatment_mean': treatment_mean,
            'control_std': control_std,
            'treatment_std': treatment_std,
            'absolute_difference': treatment_mean - control_mean,
            'relative_difference': ((treatment_mean - control_mean) / control_mean * 100) if control_mean != 0 else 0
        }
    
    def run_t_test(self):
        """Run t-test to compare means"""
        if len(self.control_data) < 2 or len(self.treatment_data) < 2:
            return None, None
        
        t_stat, p_value = stats.ttest_ind(self.control_data, self.treatment_data)
        significant = p_value < self.alpha
        
        return {
            't_statistic': t_stat,
            'p_value': p_value,
            'significant': significant
        }
    
    def calculate_effect_size(self):
        """Calculate Cohen's d effect size"""
        if len(self.control_data) < 2 or len(self.treatment_data) < 2:
            return None
        
        control_mean = np.mean(self.control_data)
        treatment_mean = np.mean(self.treatment_data)
        control_std = np.std(self.control_data)
        treatment_std = np.std(self.treatment_data)
        
        # Pooled standard deviation
        pooled_std = np.sqrt((control_std**2 + treatment_std**2) / 2)
        
        # Cohen's d
        d = (treatment_mean - control_mean) / pooled_std if pooled_std != 0 else 0
        
        return d
    
    def get_results(self):
        """Get complete test results"""
        summary = self.get_summary_stats()
        t_test = self.run_t_test()
        effect_size = self.calculate_effect_size()
        
        required_sample = self.calculate_sample_size()
        has_enough_data = (summary['control_n'] >= required_sample and 
                          summary['treatment_n'] >= required_sample)
        
        return {
            'test_name': self.name,
            'metric': self.metric_name,
            'summary': summary,
            't_test': t_test,
            'effect_size': effect_size,
            'required_sample_size': required_sample,
            'has_enough_data': has_enough_data
        }
    
    def visualize_results(self):
        """Visualize test results"""
        results = self.get_results()
        
        plt.figure(figsize=(12, 8))
        
        # Plot means with error bars
        plt.subplot(2, 2, 1)
        groups = ['Control', 'Treatment']
        means = [results['summary']['control_mean'], results['summary']['treatment_mean']]
        errors = [results['summary']['control_std'] / np.sqrt(results['summary']['control_n']) if results['summary']['control_n'] > 0 else 0,
                 results['summary']['treatment_std'] / np.sqrt(results['summary']['treatment_n']) if results['summary']['treatment_n'] > 0 else 0]
        
        plt.bar(groups, means, yerr=errors, capsize=10, color=['#1f77b4', '#ff7f0e'])
        plt.title(f'{self.metric_name} by Group')
        plt.ylabel(self.metric_name)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Plot distributions
        plt.subplot(2, 2, 2)
        if self.control_data and self.treatment_data:
            sns.kdeplot(self.control_data, label='Control', shade=True)
            sns.kdeplot(self.treatment_data, label='Treatment', shade=True)
            plt.title(f'Distribution of {self.metric_name}')
            plt.xlabel(self.metric_name)
            plt.ylabel('Density')
            plt.legend()
        
        # Plot sample size vs. required
        plt.subplot(2, 2, 3)
        sample_sizes = [results['summary']['control_n'], results['summary']['treatment_n']]
        required = results['required_sample_size']
        
        bars = plt.bar(groups, sample_sizes, color=['#1f77b4', '#ff7f0e'])
        plt.axhline(y=required, color='r', linestyle='--', label=f'Required ({required})')
        
        # Color bars based on whether they meet the requirement
        for i, bar in enumerate(bars):
            if sample_sizes[i] < required:
                bar.set_color('#d62728')  # Red for insufficient
        
        plt.title('Sample Size vs. Required')
        plt.ylabel('Sample Size')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Plot p-value
        plt.subplot(2, 2, 4)
        if results['t_test'] and 'p_value' in results['t_test']:
            p_value = results['t_test']['p_value']
            plt.bar(['p-value'], [p_value], color='g' if p_value < self.alpha else 'r')
            plt.axhline(y=self.alpha, color='r', linestyle='--', label=f'Alpha ({self.alpha})')
            plt.title('Statistical Significance')
            plt.ylabel('p-value')
            plt.ylim(0, min(1, p_value * 2) if p_value > 0 else 1)
            plt.legend()
            plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        plt.suptitle(f'A/B Test Results: {self.name}', fontsize=16)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # Print summary
        print(f"A/B Test: {self.name}")
        print(f"Metric: {self.metric_name}")
        print(f"Control: {results['summary']['control_mean']:.4f} ± {results['summary']['control_std']:.4f} (n={results['summary']['control_n']})")
        print(f"Treatment: {results['summary']['treatment_mean']:.4f} ± {results['summary']['treatment_std']:.4f} (n={results['summary']['treatment_n']})")
        print(f"Absolute difference: {results['summary']['absolute_difference']:.4f}")
        print(f"Relative difference: {results['summary']['relative_difference']:.2f}%")
        
        if results['t_test'] and 'p_value' in results['t_test']:
            print(f"p-value: {results['t_test']['p_value']:.4f}")
            print(f"Statistically significant: {results['t_test']['significant']}")
        
        if results['effect_size'] is not None:
            print(f"Effect size (Cohen's d): {results['effect_size']:.4f}")
        
        print(f"Required sample size per group: {results['required_sample_size']}")
        print(f"Has enough data: {results['has_enough_data']}")

# Demonstrate the framework with a simulated promotion test
promotion_test = ABTestFramework(
    name="10% Discount Promotion",
    metric_name="Conversion Rate",
    effect_size=0.1,
    alpha=0.05,
    power=0.8
)

# Simulate data
np.random.seed(44)

# Control group (no promotion)
control_conversion_rate = 0.05
for _ in range(1000):
    conversion = np.random.binomial(1, control_conversion_rate)
    promotion_test.add_data('control', conversion)

# Treatment group (with promotion)
treatment_conversion_rate = 0.07  # Expected 40% lift
for _ in range(1000):
    conversion = np.random.binomial(1, treatment_conversion_rate)
    promotion_test.add_data('treatment', conversion)

# Get and visualize results
promotion_test.visualize_results()

print("\nA/B testing framework implementation complete.")